In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Testicular_Cancer/GSE59520'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of known and novel germ cell cancer specific (embryonic) miRs in serum by high throughput profiling"
!Series_summary	"Very recently, a number of independent studies showed that serum levels of embryonic micro-RNA (miR) clusters 371-3 and 302abc/367 are predictive for the presence of testicular type II germ cell tumors. These miRs could be used to sensitively detect SE and EC components which are indeed known to express these miRs [1-7]. This study investigates ca 750 miRs in a high throughput approach to validate these previously identified markers and identify novel potential miR markers for testicular type II germ cell tumors.  1. Belge, G., et al., Serum levels of microRNAs miR-371-3: a novel class of serum biomarkers for testicular germ cell tumors? Eur Urol, 2012. 61(5): p. 1068-9.  2. Dieckmann, K.P., et al., MicroRNAs miR-371-3 in serum as diagnostic tools in the management of testicular germ cell tumours. Br J Cancer, 2012. 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define background info properly
background_info = {
    "!Series_title": "Identification of known and novel germ cell cancer specific (embryonic) miRs in serum by high throughput profiling",
    "!Series_summary": "Very recently, a number of independent studies showed that serum levels of embryonic micro-RNA (miR) clusters 371-3 and 302abc/367 are predictive for the presence of testicular type II germ cell tumors...",
    "!Series_overall_design": "35 serum samples were profiled for ca. 750 micro RNAs. These consisted of serum from 10 patients with non-seminomas (2 YS, 2 TE, 2 EC and 4 mixed NS which all contained EC) and 14 SE cases..."
}

# Checking dataset type from background information
if "miR" in background_info['!Series_summary']:
    is_gene_available = False
else:
    is_gene_available = True

# Checking availability and the key for trait 'Testicular_Cancer'
sample_char_dict = {
    0: ['sample type: serum from patient with epidermoid cyst', 'sample type: serum from patient with epidermis cyst', 
        'sample type: serum from patient with low grade liposarcoma', 'sample type: serum from patient with neuro endocrine tumor', 
        'sample type: serum from patient with paratesticular hemorrhage', 'sample type: serum from patient with yolk sac tumor', 
        'sample type: serum from patient with teratoma', 'sample type: serum from patient with embryonal carcinoma', 
        'sample type: serum from patient with embryonal carcinoma + yolk sac tumor', 
        'sample type: serum from patient with embryonal carcinoma + teratoma', 
        'sample type: serum from patient with seminoma', 
        'sample type: serum from normal adult testis tissue with adequate spermatogenesis (johnson score 8-10)']
}

trait_row = None
for key, values in sample_char_dict.items():
    if any('testis tissue' in value for value in values):  # Example to refine checking
        trait_row = key
        break

# Checking availability for 'Age' and 'Gender' (keys not found in sample characteristics)
age_row = None
gender_row = None

# Data type conversion functions
def convert_trait(value):
    if 'embryonal carcinoma' in value or 'yolk sac tumor' in value or 'teratoma' in value or 'seminoma' in value:
        return 1
    elif 'normal adult testis tissue' in value:
        return 0
    else:
        return None

def convert_age(value):
    return None  # Age not available

def convert_gender(value):
    return None  # Gender not available

# Save cohort info
save_cohort_info('GSE59520', './preprocessed/Testicular_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save clinical features if trait data is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Testicular_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Testicular_Cancer/trait_data/GSE59520.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Testicular_Cancer/cohort_info.json
{'GSM1438703': [None], 'GSM1438704': [None], 'GSM1438705': [None], 'GSM1438706': [None], 'GSM1438707': [None], 'GSM1438708': [None], 'GSM1438709': [None], 'GSM1438710': [None], 'GSM1438711': [1], 'GSM1438712': [1], 'GSM1438713': [1], 'GSM1438714': [1], 'GSM1438715': [1], 'GSM1438716': [1], 'GSM1438717': [1], 'GSM1438718': [1], 'GSM1438719': [1], 'GSM1438720': [1], 'GSM1438721': [1], 'GSM1438722': [1], 'GSM1438723': [1], 'GSM1438724': [1], 'GSM1438725': [1], 'GSM1438726': [1], 'GSM1438727': [1], 'GSM1438728': [1], 'GSM1438729': [1], 'GSM1438730': [1], 'GSM1438731': [1], 'GSM1438732': [1], 'GSM1438733': [1], 'GSM1438734': [1], 'GSM1438735': [0], 'GSM1438736': [0], 'GSM1438737': [0]}
